<a href="https://colab.research.google.com/github/bnarath/TF_Developer/blob/main/Transfer_Learning_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes


### What is transfer learning ?
Using another model's weight to use in our model and learn further

### Main benefits
- Can leverage an established model's learning
- Can leverage another model's architecture that is trained on similar data


### GPU Check
- !nvidia-smi
  - Tesla is Nvidia's line of GPGPU products


### Data
- We have only 75 images of each 10 categories for train and 250 images of each 10 categories for test

In [6]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## Download, Extract, Inspect

- We have only 75 images of each 10 categories for train and 250 images of each 10 categories for test

In [7]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2023-02-08 06:11:51--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 74.125.139.128, 172.217.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip.1’

10_food_classes_10_ 100%[===================>] 160.74M   135MB/s    in 1.2s    

2023-02-08 06:11:52 (135 MB/s) - ‘10_food_classes_10_percent.zip.1’ saved [168546183/168546183]



In [8]:
import zipfile
zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip', 'r')
zip_ref.extractall()
zip_ref.close()

### Check how many files?

In [19]:
import os
for dirpath, dirnames, filenames in os.walk('10_food_classes_10_percent'):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")
     

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_curry'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent

## Create dataloaders

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [23]:
IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

print("Training images:")
train_data = train_datagen.flow_from_directory(train_dir,
                                  target_size=IMAGE_SHAPE,
                                  batch_size=BATCH_SIZE,
                                  class_mode='categorical')

print("Test images:")
test_data = test_datagen.flow_from_directory(test_dir,
                                  target_size=IMAGE_SHAPE,
                                  batch_size=BATCH_SIZE,
                                  class_mode='categorical')

Training images:
Found 750 images belonging to 10 classes.
Test images:
Found 2500 images belonging to 10 classes.


## Setting up callbacks

- Callbacks ?
  - Extra functionality to add before and after training

- Popular callbacks
  - Experiment tracking with tensorboard (log the performance of multiple models, track them visually)

  - Model checkpoint - Save weights while training so that later the model can be stopped and resumed

  - Early stopping - Leave model training for an arbitrarily long time and let the model stop when it ceases to see the improvement

- How to access ?
  `tf.keras.callbacks.TensorBoard()`
  - It saves model performance to a `log_dir`
  - Logs are recorded every `update_freq=#of_epochs`
    - Logging too frequently slows down the model training

